In [15]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_predict, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np

In [2]:
#load dataset csv
dataset = pd.read_csv('C:/Users/ksmin/data_Science/dataset/Week11_DataScience-PHW_evaluation-1-datasets/knn_data.csv')

print(dataset.shape)
print()
print(dataset.info()) #no missing data
print()
print(dataset.head())

(75, 3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   longitude  75 non-null     float64
 1   latitude   75 non-null     float64
 2   lang       75 non-null     object 
dtypes: float64(2), object(1)
memory usage: 1.9+ KB
None

    longitude   latitude     lang
0  -86.750000  33.566667   Python
1  -88.250000  30.683333   Python
2 -112.016667  33.433333     Java
3 -110.933333  32.116667     Java
4  -92.233333  34.733333        R


In [13]:
#conver column lang string data type to int, float
encoder = LabelEncoder()
encoded_lang = dataset[['lang']].apply(encoder.fit_transform)

In [35]:
#Classify dataset using KNN, with K-fold, and check the best hyperparameter, use GridSearchcv, Randomsizedsearchcv
def Knn(X,y):
    split_num = 5
    k_fold = StratifiedKFold(n_splits=split_num, random_state=3, shuffle=True)

    model = KNeighborsClassifier(n_neighbors=3)
    #evaluate model accuracy.
    model.score = cross_val_predict(model, X,y.values.ravel(), cv=k_fold)
    print("Original model Accuracy : ", model.score.mean())

    #new model and will tune hyperparameter n_neighbor
    model2 = KNeighborsClassifier()
    gscv_param, gscv_best_score = knn_gscv(model2, k_fold,X,y.values.ravel())
    rscv_param, rscv_best_score = knn_rscv(model2, k_fold, X,y.values.ravel()) 
    #Find the best hyperparameter and that scoring.
    print("GridSearchCV Best parameters : {}, GridSearchCV best score : {}".format(gscv_param,gscv_best_score))
    print("RandomizedSearchCV Best parameters : {}, RandomizedSearchCV best score : {}".format(rscv_param,rscv_best_score))

    #hypertuned model2 n_neighbors = 2.
    model2 = KNeighborsClassifier(n_neighbors=2)  # Using the optimal n_neighbors value found previously by using knn_gscv,knn_rscv function.
    # Evaluate model2 accuracy
    model2.score = cross_val_predict(model2, X, y.values.ravel(),cv=k_fold)
    print("\nhypertuned model2  Accuracy:", model2.score.mean())

#GridSearchCV
def knn_gscv(model, k_fold, X,y):
    #n_neighbor is hyperparameter, n_n Neighbors should check from 1 to 75
    param_grid = {'n_neighbors' : range(1,10)}
    grid_search = GridSearchCV(model, param_grid, cv = k_fold)
    grid_search.fit(X, y)
    return grid_search.best_params_, grid_search.best_score_

#RandomizedSearchCV
def knn_rscv(model, k_fold, X,y):
    param_dist = {'n_neighbors': range(1, 10)}
    random_search = RandomizedSearchCV(model, param_dist, n_iter=10, cv=k_fold)
    random_search.fit(X, y)
    return random_search.best_params_, random_search.best_score_ 

X = dataset[['longitude', 'latitude']]
y = encoded_lang
Knn(X,y)

Original model Accuracy :  0.8
GridSearchCV Best parameters : {'n_neighbors': 2}, GridSearchCV best score : 0.5733333333333334
RandomizedSearchCV Best parameters : {'n_neighbors': 2}, RandomizedSearchCV best score : 0.5733333333333334

hypertuned model2  Accuracy: 0.6933333333333334


c:\Users\ksmin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:318: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
